In [3]:
!wget https://github.com/ant-nik/semares/raw/master/data/stereo-camera-cyl/calibrated/triangulate_P1.bin
!wget https://github.com/ant-nik/semares/raw/master/data/stereo-camera-cyl/calibrated/triangulate_P2.bin
!wget https://github.com/ant-nik/semares/raw/master/data/stereo-camera-cyl/calibrated/left_map1.bin
!wget https://github.com/ant-nik/semares/raw/master/data/stereo-camera-cyl/calibrated/left_map2.bin
!wget https://github.com/ant-nik/semares/raw/master/data/stereo-camera-cyl/calibrated/right_map1.bin
!wget https://github.com/ant-nik/semares/raw/master/data/stereo-camera-cyl/calibrated/right_map2.bin

--2024-05-11 19:26:50--  https://github.com/ant-nik/semares/raw/master/data/stereo-camera-cyl/calibrated/triangulate_P1.bin
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/calibrated/triangulate_P1.bin [following]
--2024-05-11 19:26:50--  https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/calibrated/triangulate_P1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96 [application/octet-stream]
Saving to: ‘triangulate_P1.bin.1’

triangulate_P1.bin. 100%[===================>]      96  --.-KB/s    in 0s      

2024-05-11 19

In [16]:
import pandas
import numpy
import logging
import functools
import requests
import cv2
import io
import plotly.express as plte


logger = logging.getLogger(__name__)


def load_image(payload: any) -> any:
    np_image = numpy.frombuffer(payload, numpy.uint8)
    img = cv2.imdecode(np_image, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    return img


def load_single_image(item: pandas.Series, url_template: str) -> numpy.ndarray:
    item_file = item["image"].lstrip(" ")
    url = url_template.format(item_file)
    response = requests.get(url)
    if response.status_code != 200:
        logger.error("Can't read image %s from url %s", item_file, url)
    return pandas.Series([load_image(response.content), url],
                         index=["image_data", "url"])

    import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt


def calculate_sift_descriptors(image: any) -> tuple[any, any]:
    sift = cv2.SIFT_create()
    # find the keypoints and descriptors with SIFT
    return sift.detectAndCompute(image, None)


def calculate_orb_descriptors(image: numpy.ndarray) -> tuple[any, any]:
    orb = cv.ORB_create()
    # find the keypoints with ORB
    kp = orb.detect(image, None)
    # compute the descriptors with ORB
    kp, des = orb.compute(image, kp)

    return kp, des


def calculate_match_points(
        kp1: numpy.ndarray,
        des1: numpy.ndarray,
        kp2: numpy.ndarray,
        des2: numpy.ndarray) -> tuple[list, list]:
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    return pts1, pts2


def calculate_fundamental_matrix(
    p1: numpy.ndarray, p2: numpy.ndarray
) -> tuple[numpy.ndarray, numpy.ndarray]:
    p1 = numpy.int32(p1)
    p2 = numpy.int32(p2)
    return cv2.findFundamentalMat(p1, p2, cv2.FM_LMEDS)


def draw_matched_points(
    left_image: numpy.ndarray,
    right_image: numpy.ndarray,
    left_points: numpy.ndarray,
    right_points: numpy.ndarray
) -> None:
    canvas_shape = list(left_image.shape)
    canvas_shape[1] = canvas_shape[1]*2
    canvas = numpy.zeros(canvas_shape)
    canvas[:, 0:left_image.shape[1], :] = left_image
    canvas[:, left_image.shape[1]:, :] = right_image
    for p_left, p_right in zip(left_points, right_points):
        color = tuple(numpy.random.randint(0, 255, 3).tolist())
        left_point = tuple(map(int, p_left))
        canvas = cv2.circle(canvas, left_point, 5, color,-1)
        p_right_shifted = list(p_right)
        p_right_shifted[0] = p_right_shifted[0] + left_image.shape[1]
        right_point = tuple(map(int, p_right_shifted))
        canvas = cv2.circle(canvas, right_point, 5, color,-1)
        canvas = cv2.line(canvas, left_point, right_point, (0, 255, 0), 1)
    fig = plte.imshow(canvas)
    fig.show()


def draw_image_sequence(*args, axis=1) -> numpy.ndarray:
    if len(args) == 0:
        return numpy.ndarray()

    w = args[0].shape[1]
    h = args[0].shape[0]
    canvas_shape = list(args[0].shape)
    canvas_shape[axis] = canvas_shape[axis]*len(args)
    canvas = numpy.zeros(canvas_shape)
    for i in range(0, len(args)):
        if axis == 1:
            canvas[:, i*w:(i + 1)*w, :] = args[i]
        else:
            canvas[i*w:(i + 1)*w, :, :] = args[i]

    return canvas

In [4]:
import numpy


P1 = numpy.fromfile("triangulate_P1.bin")
P2 = numpy.fromfile("triangulate_P2.bin")
left_map1 = numpy.fromfile("left_map1.bin")
left_map2 = numpy.fromfile("left_map2.bin")
right_map1 = numpy.fromfile("right_map1.bin")
right_map2 = numpy.fromfile("right_map2.bin")

In [23]:
left_map1

array([1.25185444e-308, 1.25185656e-308, 1.25186081e-308, ...,
       5.97588909e-300, 5.97589479e-300, 5.97590048e-300])

In [5]:
!wget -O samples.csv https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/position.csv

--2024-05-11 19:35:46--  https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/position.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1728 (1.7K) [text/plain]
Saving to: ‘samples.csv’

samples.csv         100%[===================>]   1.69K  --.-KB/s    in 0s      

2024-05-11 19:35:46 (20.9 MB/s) - ‘samples.csv’ saved [1728/1728]



In [9]:
base_url = "https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/{}"

dataframe = pandas.read_csv("samples.csv", sep=',')
dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            load_single_image,
            url_template=base_url),
        axis=1)
)

In [15]:
left_image_1 = dataframe[dataframe["image"] == "image36_l.jpg"]["image_data"].iloc[0]
right_image_1 = dataframe[dataframe["image"] == "image36_r.jpg"]["image_data"].iloc[0]

In [19]:
plte.imshow(draw_image_sequence(left_image_1, right_image_1))

In [33]:
left_map2.shape, left_map2.reshape(480, 640).shape

ValueError: cannot reshape array of size 76800 into shape (480,640)

In [34]:
left_map1.shape, left_map1.reshape(480, 640, -1).shape

ValueError: cannot reshape array of size 153600 into shape (480,640,newaxis)

In [20]:
left_rimage_2 = cv2.remap(
    left_image_1,
    left_map1, left_map2, cv2.INTER_LINEAR)
right_rimage_2 = cv2.remap(
    right_image_1,
    right_map1, right_map2, cv2.INTER_LINEAR)

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/imgwarp.cpp:1710: error: (-215:Assertion failed) _map2.empty() || (_map2.size() == _map1.size()) in function 'remap'


In [ ]:
kp1, des1 = calculate_sift_descriptors(image=left_rimage_2)
kp2, des2 = calculate_sift_descriptors(image=right_rimage_2)

left_pts, right_pts = calculate_match_points(
    kp1=kp1, des1=des1,
    kp2=kp2, des2=des2)